In [528]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from thermography_dataset_one_layer import ThermDataset
from sklearn.model_selection import train_test_split
%matplotlib inline

In [529]:
args = {'lr':0.01,
        'epochs':1000,
        'noise':0,
        'train size':0.7,
        'spec scale':10**12
        }

num_layers = 11

In [530]:
def df_to_tensor(df):
    return torch.tensor(df.values, dtype=torch.float32)

# Preparing Data

In [531]:
df = pd.read_excel('wide_range.xlsx')

In [532]:
X = df.iloc[:,11:]
y = df.iloc[:,:11]

X = X.apply(lambda x: x*args['spec scale'])


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=args['train size'], random_state=101)


X_train.index = range(len(X_train))
X_test.index = range(len(X_test))

In [533]:
X_train

,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,0.047328,0.044030,0.062621,0.050627,0.071193,0.068130,0.101914,0.075729,0.108325,0.133987,...,3.652357,3.706860,3.754378,3.791491,3.816109,3.819172,3.783524,3.692427,3.557224,3.432674
1,0.038770,0.035673,0.050047,0.040308,0.056641,0.054457,0.082382,0.062676,0.091149,0.114388,...,2.320425,2.355479,2.389728,2.419447,2.441954,2.450765,2.434611,2.382484,2.301435,2.226764
2,0.017471,0.015108,0.019534,0.015249,0.021160,0.020663,0.033139,0.028438,0.044417,0.059315,...,1.783106,1.818651,1.850644,1.877459,1.898089,1.907974,1.898374,1.860607,1.800060,1.744286
3,0.024575,0.020517,0.025340,0.019283,0.026321,0.025631,0.041952,0.037789,0.060454,0.082459,...,4.617065,4.680924,4.734778,4.774832,4.798921,4.795918,4.744459,4.623791,4.448374,4.286812
4,0.012262,0.010760,0.014171,0.011194,0.015674,0.015384,0.024641,0.020957,0.032692,0.043657,...,1.639148,1.670154,1.699314,1.724537,1.744379,1.754398,1.746490,1.712636,1.657755,1.607205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,0.008816,0.007552,0.009605,0.007467,0.010369,0.010237,0.016819,0.015040,0.024138,0.032982,...,1.737692,1.768599,1.798230,1.824086,1.844388,1.854326,1.845326,1.808934,1.750374,1.696432
1536,0.022660,0.019827,0.026071,0.020487,0.028498,0.027720,0.043868,0.036680,0.056381,0.074280,...,1.875067,1.911222,1.943955,1.971422,1.992454,2.002216,1.991539,1.951336,1.887281,1.828272
1537,0.054368,0.049658,0.069249,0.055489,0.077627,0.074266,0.111904,0.084833,0.122915,0.153867,...,5.344826,5.409761,5.465227,5.505966,5.528740,5.520424,5.456460,5.313137,5.107256,4.917666
1538,0.017072,0.014896,0.019520,0.015351,0.021409,0.020917,0.033378,0.028303,0.043981,0.058593,...,2.570995,2.615463,2.655006,2.687193,2.710532,2.718533,2.698864,2.639391,2.547993,2.463787


In [534]:
y_train

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,layer 11
805,370.341309,328.561339,326.246196,325.960978,337.115108,338.011684,329.244973,338.854172,334.003205,331.473263,344.660386
1798,358.717780,321.142036,335.121747,323.227587,334.855686,331.980768,331.899256,332.369302,336.197365,344.170862,318.387290
193,303.578596,296.012106,320.139477,331.612634,343.149874,341.599287,327.050020,312.050438,300.909822,290.052707,305.241303
162,281.108362,326.802138,327.622403,330.956774,328.315642,325.316098,330.881045,323.297827,335.718231,335.832163,359.882238
1211,300.756094,302.244654,302.319500,307.933527,316.877701,315.324771,318.254083,318.875332,313.324653,307.896394,301.072836
...,...,...,...,...,...,...,...,...,...,...,...
599,274.142753,312.028924,305.203144,306.792005,301.311853,291.127893,298.271076,303.973224,308.318167,319.322458,303.811769
1599,318.512434,351.712729,343.852709,337.011663,325.408620,325.728092,318.063111,309.798561,314.474813,297.388722,307.688462
1361,370.224132,340.547318,330.792273,325.245505,318.753576,321.749459,328.052407,338.216759,343.711867,361.540581,369.965830
1547,308.722907,320.029113,305.872919,303.670763,308.517581,320.283005,326.395634,328.204725,333.495631,311.017505,324.166657


# Creating Models

In [535]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(input_size, 45)
        self.lin2 = nn.Linear(45, 60)
        self.lin3 = nn.Linear(60, 75)
        self.lin4 = nn.Linear(75, 60)
        self.lin_fin = nn.Linear(60, output_size)


    def forward(self, x):
        x = F.leaky_relu(self.lin1(x))
        x = F.leaky_relu(self.lin2(x))
        x = F.leaky_relu(self.lin3(x))
        x = F.leaky_relu(self.lin4(x))
        x = self.lin_fin(x)
        return x

In [536]:
input_size = len(X_train.columns)
output_size = 1

print(f' input size: {input_size}, output size: {output_size}')

models = {}
for i in range(1, 12):
    models[f'model{i}'] = Net(input_size + (-i+11), output_size)

print(models)

 input size: 86, output size: 1
{'model1': Net(
  (lin1): Linear(in_features=96, out_features=45, bias=True)
  (lin2): Linear(in_features=45, out_features=60, bias=True)
  (lin3): Linear(in_features=60, out_features=75, bias=True)
  (lin4): Linear(in_features=75, out_features=60, bias=True)
  (lin_fin): Linear(in_features=60, out_features=1, bias=True)
), 'model2': Net(
  (lin1): Linear(in_features=95, out_features=45, bias=True)
  (lin2): Linear(in_features=45, out_features=60, bias=True)
  (lin3): Linear(in_features=60, out_features=75, bias=True)
  (lin4): Linear(in_features=75, out_features=60, bias=True)
  (lin_fin): Linear(in_features=60, out_features=1, bias=True)
), 'model3': Net(
  (lin1): Linear(in_features=94, out_features=45, bias=True)
  (lin2): Linear(in_features=45, out_features=60, bias=True)
  (lin3): Linear(in_features=60, out_features=75, bias=True)
  (lin4): Linear(in_features=75, out_features=60, bias=True)
  (lin_fin): Linear(in_features=60, out_features=1, bias=T

# Training

In [537]:
learning_rate = args['lr']
criterion = nn.L1Loss()
num_epochs = args['epochs']

In [538]:
loss_lists = {}
losses = {}
inputs = df_to_tensor(X_train)
prev_pred = None


for i in range(num_layers):
    layer = 11 - i
    print(f'Layer {layer}')

    # define loss lists
    loss_list = f'loss_list{layer}'
    loss_lists[loss_list] = []

    layer_loss_name = f'loss{layer}'
    losses[layer_loss_name] = 0

    # define the model corresponding to the given layer
    model = models[f'model{layer}']

    # define optimizer corresponding to the given model
    optimizer = torch.optim.SGD(models[f'model{layer}'].parameters(), lr=learning_rate)

    # add previous predictions to inputs if necessary

    if i > 0:
        pred_temp_new = pred_temp.clone()
        prev_pred = pred_temp_new.reshape(-1, 1)
        inputs = torch.cat([inputs, prev_pred], dim=1)

    print(inputs)
    print(f'inputs shape: {inputs.shape}')
    print(f'model input shape: {model}')

    # get expected_temp appropriately
    expected_temp = df_to_tensor(y_train.iloc[:,layer-1]).reshape(-1,1)

    # begin training loop
    for epoch in range(num_epochs):
        # empty gradients
        optimizer.zero_grad()

        # forward pass
        pred_temp = model(inputs)

        # calculate loss and add to loss list
        loss = criterion(pred_temp, expected_temp)
        loss_lists[f'loss_list{layer}'].append(loss.item())

        # backward pass
        loss.backward()
        
        # update
        optimizer.step()

        # print losses
        if (epoch+1) % int(args['epochs']/10) == 0:
            print(f'epoch: {epoch+1}, loss = {loss}')



Layer 11
tensor([[0.0473, 0.0440, 0.0626,  ..., 3.6924, 3.5572, 3.4327],
        [0.0388, 0.0357, 0.0500,  ..., 2.3825, 2.3014, 2.2268],
        [0.0175, 0.0151, 0.0195,  ..., 1.8606, 1.8001, 1.7443],
        ...,
        [0.0544, 0.0497, 0.0692,  ..., 5.3131, 5.1073, 4.9177],
        [0.0171, 0.0149, 0.0195,  ..., 2.6394, 2.5480, 2.4638],
        [0.0214, 0.0197, 0.0276,  ..., 3.4464, 3.3216, 3.2066]])
inputs shape: torch.Size([1540, 86])
model input shape: Net(
  (lin1): Linear(in_features=86, out_features=45, bias=True)
  (lin2): Linear(in_features=45, out_features=60, bias=True)
  (lin3): Linear(in_features=60, out_features=75, bias=True)
  (lin4): Linear(in_features=75, out_features=60, bias=True)
  (lin_fin): Linear(in_features=60, out_features=1, bias=True)
)
epoch: 100, loss = 321.6754455566406
epoch: 200, loss = 320.94586181640625
epoch: 300, loss = 319.7139892578125
epoch: 400, loss = 318.9426574707031
epoch: 500, loss = 317.6376037597656
epoch: 600, loss = 316.9668273925781


/Users/aidenkarpf/anaconda3/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "/Users/aidenkarpf/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/aidenkarpf/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/aidenkarpf/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/aidenkarpf/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/aidenkarpf/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/Users/aidenkarpf/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_for

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.